In [25]:
import pandas as pd

In [26]:
class_list = ['NOT_TRIP', 'TRIP']

In [27]:
df = pd.read_csv("datasets/bert_ds.csv", delimiter=';')

In [28]:
del df['origin']
del df['destination']
del df['detours']
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [29]:
df['pred_class'] = df.apply(lambda x: 1 if x["is_trip"] == 1 else 0,axis=1)
del df['is_trip']

In [30]:
df.head()

,text,pred_class
0,je veux partir de jussy en direction de caucal...,1
1,je veux un train entre virlet et st goussaud,1
2,jaimerais me rendre a brainville depuis chanac,1
3,je voudrais aller a loriol sur drome depuis so...,1
4,je suis a haudricourt et je retourne a talange...,1


In [31]:
from sklearn.model_selection import train_test_split
 
train_df, test_df = train_test_split(df, test_size=0.10)
 
print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (1800, 2)
test shape:  (200, 2)


In [32]:
from simpletransformers.classification import ClassificationModel

In [33]:
train_args ={
    "reprocess_input_data": True,
    "fp16":False,
    "num_train_epochs": 4
}

In [34]:
model = ClassificationModel(
    "bert", "outputs",
    num_labels=2,
    args=train_args,
    use_cuda=False,
)

In [36]:
model.train_model(train_df)

/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v

(900, 0.02305656862285812)

In [37]:
from sklearn.metrics import f1_score, accuracy_score
 
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
 
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
0it [00:00, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:03,  3.73s/it]
Running Evaluation: 100%|██████████| 2/2 [00:03<00:00,  1.97s/it]


In [38]:
result

{'mcc': 1.0,
 'accuracy': 1.0,
 'f1_score': 1.0,
 'tp': 91,
 'tn': 109,
 'fp': 0,
 'fn': 0,
 'auroc': 1.0,
 'auprc': 1.0,
 'f1': 1.0,
 'acc': 1.0,
 'eval_loss': 1.5371305380540434e-05}

In [40]:
test = "j'aimerais aller a strasbourg"
predictions, raw_outputs = model.predict([test])
print(class_list[predictions[0]])

0it [00:00, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:03,  3.05s/it]
100%|██████████| 1/1 [00:00<00:00, 12.42it/s]

TRIP


In [10]:
import os
import tarfile

In [47]:
def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [48]:
pack_model('outputs', 'bert-fr-trip')

In [49]:
ds_val = pd.read_csv("datasets/bert_ds_val.csv", delimiter=';')

del ds_val['origin']
del ds_val['destination']
del ds_val['detours']
ds_val.drop(columns=ds_val.columns[0], axis=1, inplace=True)

ds_val['pred_class'] = ds_val.apply(lambda x: 1 if x["is_trip"] == 1 else 0,axis=1)
del ds_val['is_trip']

In [50]:
ds_val = ds_val.sample(frac=1)

In [51]:
ds_val

,text,pred_class
1,je suis a la chapelle en vercors et jaimerais ...,1
3,voyage de la bruguiere jusqua auriac,1
7,je prefere la plage de clave a celle de st mar...,0
0,je planifie un voyage entre beaumont hamel et ...,1
4,je prevois de voyager de spincourt a marigny d...,1
6,viel arcy a une ambiance tres cosmopolite,0
5,jai entendu dire que mont st jean organise un ...,0
9,les couchers de soleil a dommartin le coq sont...,0
2,voyage de lespielle jusqua ste nathalene,1
8,les festivals a prades sont toujours incroyables,0


In [52]:
result = 0
for _, row in ds_val.iterrows():
    text = row['text']
    real_val = row['pred_class']
    predictions, _ = model.predict([text])
    print(f'text: {text}')
    print(f'\tPREDICTED: {class_list[predictions[0]]}')
    print(f'\tREAL: {class_list[real_val]}')
    if(class_list[predictions[0]] == class_list[real_val]):
        result += 1

0it [00:00, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:03,  3.04s/it]
100%|██████████| 1/1 [00:00<00:00, 14.17it/s]

text: je suis a la chapelle en vercors et jaimerais aller a st meslin du bosc
	PREDICTED: TRIP
	REAL: TRIP



0it [00:00, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:03,  3.02s/it]
100%|██████████| 1/1 [00:00<00:00, 13.20it/s]

text: voyage de la bruguiere jusqua auriac
	PREDICTED: TRIP
	REAL: TRIP



0it [00:00, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:02,  2.98s/it]
100%|██████████| 1/1 [00:00<00:00, 12.03it/s]

...

text: les couchers de soleil a dommartin le coq sont magnifiques
	PREDICTED: NOT_TRIP
	REAL: NOT_TRIP



0it [00:00, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:03,  3.03s/it]
100%|██████████| 1/1 [00:00<00:00, 12.70it/s]

text: voyage de lespielle jusqua ste nathalene
	PREDICTED: TRIP
	REAL: TRIP



0it [00:00, ?it/s]/Users/gurkan/coding/python/navify_nlp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1it [00:03,  3.07s/it]
100%|██████████| 1/1 [00:00<00:00, 11.65it/s]

text: les festivals a prades sont toujours incroyables
	PREDICTED: NOT_TRIP
	REAL: NOT_TRIP


In [53]:
print(f'Number of rows in val: {len(ds_val)}')
print(f'Accuracy: {(result * 100) / len(ds_val):.2f}%')

Number of rows in val: 10
Accuracy: 100.00%
